In [0]:
!pip install xgboost

import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
import statistics
from sklearn.metrics import matthews_corrcoef

hgg1=pd.read_csv("")#Path to csv
lgg1=pd.read_csv("")

lgg1["GRADE"]=0
hgg1["GRADE"]=1
hgg= hgg1.sample(frac=1, random_state=42).reset_index(drop=True)
lgg= lgg1.sample(frac=1, random_state=42).reset_index(drop=True)

!pip install ReliefF
from ReliefF import ReliefF


auc_lr=[]
pre_lr=[]
rec_lr=[]
acc_lr=[]
f1_lr=[]
auc_lrt=[]
pre_lrt=[]
rec_lrt=[]
acc_lrt=[]
f1_lrt=[]

master=pd.concat([lgg, hgg])
master= master.sample(frac=1, random_state=42).reset_index(drop=True)
Y=master["GRADE"]
master= master.drop("GRADE", axis=1)
master = (master - master.min()) / (master.max() - master.min())

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(master, Y):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test = master.iloc[train_index], master.iloc[test_index]
    Y_train, Y_test= Y.iloc[train_index], Y.iloc[test_index]

    X_train= X_train.values
    X_test= X_test.values
    Y_train= Y_train.values
    Y_test= Y_test.values

    clf = make_pipeline(ReliefF(n_features_to_keep=55, n_neighbors=25),
                    XGBClassifier(n_estimators=500, max_depth=1, learning_rate=0.08, min_child_weight=1))
    
    clf.fit(X_train, Y_train)

    pred = clf.predict(X_test)
    pre_l = precision_score(Y_test, pred)
    rec_l = recall_score(Y_test, pred)
    acc_l = accuracy_score(Y_test, pred)
    probs = clf.predict_proba(X_test)
    probs = probs[:, 1]
    auc_l = roc_auc_score(Y_test, probs)
    f1_l=f1_score(Y_test, pred)

    pred = clf.predict(X_train)
    pre_lt = precision_score(Y_train, pred)
    rec_lt = recall_score(Y_train, pred)
    acc_lt = accuracy_score(Y_train, pred)
    probs = clf.predict_proba(X_train)
    probs = probs[:, 1]
    auc_lt = roc_auc_score(Y_train, probs)
    f1_lt = f1_score(Y_train, pred)


    auc_lr.append(auc_l)
    pre_lr.append(pre_l)
    rec_lr.append(rec_l)
    acc_lr.append(acc_l)
    f1_lr.append(f1_l)

    auc_lrt.append(auc_lt)
    pre_lrt.append(pre_lt)
    rec_lrt.append(rec_lt)
    acc_lrt.append(acc_lt)
    f1_lrt.append(f1_lt)


print("TRAIN:")
print("PRE:", statistics.mean(pre_lrt))
print("REC:", statistics.mean(rec_lrt))
print("ACC:", statistics.mean(acc_lrt))
print("AUC:", statistics.mean(auc_lrt))
print("F1:", statistics.mean(f1_lrt))

print("TEST:")
print("PRE:", statistics.mean(pre_lr))
print("REC:", statistics.mean(rec_lr))
print("ACC:", statistics.mean(acc_lr))
print("AUC:", statistics.mean(auc_lr))
print("F1:", statistics.mean(f1_lr))